In [46]:
import numpy as np
import pandas as pd
import pickle

In [47]:
!pip install datasets

In [48]:
import nltk

In [49]:
data =pd.DataFrame()

In [50]:
with open("lemmatized_tweets (1).pkl","rb") as file_handle:
    data1 = pickle.load(file_handle)

In [51]:
data['text'] = data1

In [52]:
data['text']

0          [twitpic, com, zl, awww, bummer, shoulda, get,...
1          [upset, update, facebook, texte, cry, result, ...
2                                [manage, save, rest, bound]
3                            [body, feel, itchy, like, fire]
4                                              [behave, mad]
                                 ...                        
1599995                           [wake, school, good, feel]
1599996      [thewdb, com, cool, hear, old, walt, interview]
1599997                 [ready, mojo, makeover, ask, detail]
1599998    [happy, th, birthday, boo, alll, time, tupac, ...
1599999    [happy, charitytuesday, thenspcc, sparkscharit...
Name: text, Length: 1600000, dtype: object

In [53]:
converted_raw_text = list(data["text"].apply(lambda x: " ".join(x)))

In [54]:
converted_raw_text = list(filter(lambda x: len(x) > 1,converted_raw_text))

In [55]:
first_100000_tweets = converted_raw_text[:100000]

# Save to a text file
with open('first_100000_tweets.txt', 'w', encoding='utf-8') as file:
    for tweet in first_100000_tweets:
        file.write(tweet + '\n')

print("First 100000 tweets have been saved to 'first_100000_tweets.txt'.")

First 100000 tweets have been saved to 'first_100000_tweets.txt'.


In [56]:
import datasets
dset = datasets.load_dataset('text', data_files={"train":'first_100000_tweets.txt'})

Generating train split: 100000 examples [00:00, 2947031.75 examples/s]


In [57]:
def tokenize_tweets(single_row):
    single_row['tokenized-tweets'] = single_row["text"].split()
    return single_row

In [58]:
dset = dset.map(tokenize_tweets)

Map: 100%|██████████| 100000/100000 [00:03<00:00, 32822.70 examples/s]


In [ ]:
dset['train']['tokenized-tweets']

In [60]:
!pip install ngrams

In [61]:
from nltk.util import ngrams

def convert_to_trigrams(single_row):
    single_row["tri-grams"] = list(ngrams(single_row['tokenized-tweets'],n=3))
    return single_row

In [62]:
dset['train'] = dset['train'].map(convert_to_trigrams)

Map: 100%|██████████| 100000/100000 [00:05<00:00, 18296.73 examples/s]


In [63]:
dset

DatasetDict({
    train: Dataset({
        features: ['text', 'tokenized-tweets', 'tri-grams'],
        num_rows: 100000
    })
})

In [64]:
converted_raw_text[0]

'twitpic com zl awww bummer shoulda get david carr day'

In [65]:
vocabulary = set()
for raw_text in converted_raw_text[:100000]:
    vocabulary.update(raw_text.split())

In [66]:
len(vocabulary)

43953

In [ ]:
vocabulary

In [68]:
vocab2idx = dict(zip(vocabulary, range(len(vocabulary))))

In [ ]:
vocab2idx

In [80]:
def convert_to_bigrams(single_row):
    center_token_token_pairs = list()
    for single_trigrams in single_row["tri-grams"]:
        bigrams = list()
        bigrams.append([vocab2idx[single_trigrams[1]], vocab2idx[single_trigrams[0]]])
        bigrams.append([vocab2idx[single_trigrams[1]], vocab2idx[single_trigrams[2]]])
        center_token_token_pairs.append(bigrams)
    single_row['tri-grams'] = center_token_token_pairs
    
    return single_row

In [81]:
dset['train'] = dset['train'].map(convert_to_bigrams)

Map: 100%|██████████| 100000/100000 [00:10<00:00, 9793.37 examples/s]


In [84]:
dset['train'][0]

{'text': 'twitpic com zl awww bummer shoulda get david carr day',
 'tokenized-tweets': ['twitpic',
  'com',
  'zl',
  'awww',
  'bummer',
  'shoulda',
  'get',
  'david',
  'carr',
  'day'],
 'tri-grams': [[[9652, 5980], [9652, 22010]],
  [[22010, 9652], [22010, 23609]],
  [[23609, 22010], [23609, 9043]],
  [[9043, 23609], [9043, 33850]],
  [[33850, 9043], [33850, 20179]],
  [[20179, 33850], [20179, 31273]],
  [[31273, 20179], [31273, 14319]],
  [[14319, 31273], [14319, 19647]]]}

In [85]:
input_token_target_token_pairs = list()
for single_tweet_bigrams in dset['train']['tri-grams']:
    for bigrams_list in single_tweet_bigrams:
        input_token_target_token_pairs.append(bigrams_list[0])
        input_token_target_token_pairs.append(bigrams_list[1])
        

In [89]:
input_token_target_token_pairs[:4]

[[9652, 5980], [9652, 22010], [22010, 9652], [22010, 23609]]